In [17]:
import numpy
import tables
from tables import *

class Particle(tables.IsDescription):
    name      = tables.StringCol(16)   # 16-character String
    idnumber  = tables.Int64Col()      # Signed 64-bit integer
    ADCcount  = tables.UInt16Col()     # Unsigned short integer
    TDCcount  = tables.UInt8Col()      # unsigned byte
    grid_i    = tables.Int32Col()      # 32-bit integer
    grid_j    = tables.Int32Col()      # 32-bit integer
    pressure  = tables.Float32Col()    # float  (single-precision)
    energy    = tables.Float64Col()    # double (double-precision)


In [3]:
h5file = tables.open_file("tutorial1.h5", mode = "w", title = "Test file")

In [4]:
group = h5file.create_group("/", 'detector', 'Detector information')

In [5]:
table = h5file.create_table(group, 'readout', Particle, "Readout example")

In [6]:
h5file

File(filename=tutorial1.h5, title='Test file', mode='w', root_uep='/', filters=Filters(complevel=0, shuffle=False, fletcher32=False, least_significant_digit=None))
/ (RootGroup) 'Test file'
/detector (Group) 'Detector information'
/detector/readout (Table(0,)) 'Readout example'
  description := {
  "ADCcount": UInt16Col(shape=(), dflt=0, pos=0),
  "TDCcount": UInt8Col(shape=(), dflt=0, pos=1),
  "energy": Float64Col(shape=(), dflt=0.0, pos=2),
  "grid_i": Int32Col(shape=(), dflt=0, pos=3),
  "grid_j": Int32Col(shape=(), dflt=0, pos=4),
  "idnumber": Int64Col(shape=(), dflt=0, pos=5),
  "name": StringCol(itemsize=16, shape=(), dflt=b'', pos=6),
  "pressure": Float32Col(shape=(), dflt=0.0, pos=7)}
  byteorder := 'little'
  chunkshape := (1394,)

In [7]:
particle = table.row

In [8]:
for i in range(10):
    particle['name']  = 'Particle: %6d' % (i)
    particle['TDCcount'] = i % 256
    particle['ADCcount'] = (i * 256) % (1 << 16)
    particle['grid_i'] = i
    particle['grid_j'] = 10 - i
    particle['pressure'] = float(i*i)
    particle['energy'] = float(particle['pressure'] ** 4)
    particle['idnumber'] = i * (2 ** 34)
    # Insert a new particle record
    particle.append()

In [9]:
table.flush()  # Write to disk (RAM is freed up)

In [10]:
table = h5file.root.detector.readout
pressure = [x['pressure'] for x in table.iterrows() if x['TDCcount'] > 3 and 20 <= x['pressure'] < 50]
pressure

[25.0, 36.0, 49.0]

In [11]:
# Faster in-kernel and indexed queries
names = [ x['name'] for x in table.where("""(TDCcount > 3) & (20 <= pressure) & (pressure < 50)""") ]
names


[b'Particle:      5', b'Particle:      6', b'Particle:      7']

Note: A special care should be taken when the query condition includes string literals. Indeed Python 2 string literals are string of bytes while Python 3 strings are unicode objects.

In [12]:
# Python 2:
condition = '(name == "Particle: 5") | (name == "Particle: 7")'
# Python 3:
condition = '(name == b"Particle: 5") | (name == b"Particle: 7")'
for record in table.where(condition):  # TypeError in Python3
    print(record)

Creating new array objects
==

In [13]:
gcolumns = h5file.create_group(h5file.root, "columns", "Pressure and Name")

In [15]:
h5file.create_array(gcolumns, 'pressure', numpy.array(pressure), "Pressure column selection")

/columns/pressure (Array(3,)) 'Pressure column selection'
  atom := Float64Atom(shape=(), dflt=0.0)
  maindim := 0
  flavor := 'numpy'
  byteorder := 'little'
  chunkshape := None

We already know the first two parameters of the File.create_array() methods (these are the same as the first two in create_table): they are the parent group where Array will be created and the Array instance name. The third parameter is the object we want to save to disk. In this case, it is a NumPy array that is built from the selection list we created before. The fourth parameter is the title.

In [16]:
h5file.create_array(gcolumns, 'name', names, "Name column selection")

/columns/name (Array(3,)) 'Name column selection'
  atom := StringAtom(itemsize=16, shape=(), dflt=b'')
  maindim := 0
  flavor := 'python'
  byteorder := 'irrelevant'
  chunkshape := None

In [24]:
print(h5file)

tutorial1.h5 (File) 'Test file'
Last modif.: 'Thu Sep  8 08:31:23 2016'
Object Tree: 
/ (RootGroup) 'Test file'
/columns (Group) 'Pressure and Name'
/columns/name (Array(3,)) 'Name column selection'
/columns/pressure (Array(3,)) 'Pressure column selection'
/detector (Group) 'Detector information'
/detector/readout (Table(10,)) 'Readout example'



In [29]:
#Close a file
h5file.close()

SyntaxError: invalid syntax (<ipython-input-29-3df4304954ab>, line 5)

Read it on the command line:
$ h5ls -rd tutorial1.h5

$ ptdump tutorial1.h5
options -v - d

Browsing the object tree
==

In [30]:
h5file = open_file("tutorial1.h5", "a")

In [31]:
print(h5file)

tutorial1.h5 (File) 'Test file'
Last modif.: 'Thu Sep  8 09:02:11 2016'
Object Tree: 
/ (RootGroup) 'Test file'
/columns (Group) 'Pressure and Name'
/columns/name (Array(3,)) 'Name column selection'
/columns/pressure (Array(3,)) 'Pressure column selection'
/detector (Group) 'Detector information'
/detector/readout (Table(10,)) 'Readout example'

